!pip install sklearn --upgrade
!pip install python-swiftclient
!pip install python-keystoneclient
! cp /usr/local/src/data-connectors-1.4.1/db2jcc4-10.5.0.6.jar /gpfs/fs01/user/s29c-694fb46c266841-c78410792144/.local/lib/python2.7/site-packages/ibmdbpy/db2jcc4.jar

### The above cell should be run 1 time only. Once complete, Restart the Kernel.


Load your BigSql and ObjectStorage credentials below


In [23]:
# The code was removed by DSX for sharing.

In [24]:
import swiftclient
import numpy
import os
from sklearn.ensemble import RandomForestClassifier
from ibmdbpy import IdaDataBase
from ibmdbpy import IdaDataFrame
from collections import defaultdict
from sklearn.preprocessing import Imputer
from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder
from tempfile import mkdtemp
from jaydebeapi import _DEFAULT_CONVERTERS, _java_to_py
_DEFAULT_CONVERTERS.update({'BIGINT': _java_to_py('longValue')})

Next, let's open a connection to our database and pull in our sample data

In [25]:
bsql = credentials_1
objs = credentials_2

In [26]:
dsn_hostname =bsql['host'] 
dsn_port = bsql['port'] 
dsn_database = bsql['database']  
dsn_protocol = "TCPIP"          
dsn_uid = bsql['user']    
dsn_pwd = bsql['password']  

connection_string='jdbc:db2://'+dsn_hostname+':'+dsn_port+'/'+dsn_database+':user='+dsn_uid+';password='+dsn_pwd+";" 
idaf = IdaDataFrame(IdaDataBase(dsn=connection_string), 'BIGSQL.MOVIES', indexer="ID").as_dataframe();
print (idaf.head())
    

      title  netflix_id  customerId  GENDER  SENIORCITIZEN  DEPENDENTS  \
0  Rounders    17236920  0089-IIQKO  Female              0           1   
1  Rounders    17236920  0264-CNITK  Female              0           1   
2  Rounders    17236920  2091-GPPIQ  Female              0           1   
3   Payback    18957710  3018-TFTSU    Male              0           0   
4   Payback      343044  0004-TLHLJ    Male              0           0   

   TENURE  PAPERLESSBILLING              PAYMENTMETHOD  MONTHLYCHARGES  
0      39                 1    Credit card (automatic)           99.95  
1      71                 0  Bank transfer (automatic)           20.10  
2      72                 1           Electronic check           78.95  
3      12                 0  Bank transfer (automatic)           81.70  
4       4                 1           Electronic check           73.90  


In [27]:
le=LabelEncoder()
d = defaultdict(LabelEncoder)
for col in idaf:
       if idaf[col].dtypes=='object':
            idaf[col] = idaf.apply(lambda x: d[idaf[col].name].fit_transform(idaf[col]))

print (idaf.head())


   title  netflix_id  customerId  GENDER  SENIORCITIZEN  DEPENDENTS  TENURE  \
0     24    17236920          39       0              0           1      39   
1     24    17236920          72       0              0           1      71   
2     24    17236920         147       0              0           1      72   
3     15    18957710         155       1              0           0      12   
4     15      343044           2       1              0           0       4   

   PAPERLESSBILLING  PAYMENTMETHOD  MONTHLYCHARGES  
0                 1              1           99.95  
1                 0              0           20.10  
2                 1              2           78.95  
3                 0              0           81.70  
4                 1              2           73.90  


In [28]:
dataSet = Imputer().fit_transform(idaf.values)

Now we will run a Random Forest algorithm on the data

In [29]:
rf_c= RandomForestClassifier(n_estimators = 100, n_jobs = 2)
target = [x[0] for x in dataSet]
trainingData = [x[3:10] for x in dataSet]
rf_c.fit(trainingData, target)



RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=100, n_jobs=2, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

Let's test our model

In [30]:
inpredict = [0,1,0,4,0,3,30]
inpredict = numpy.array(inpredict).reshape(1, (len(inpredict)))

d["title"].inverse_transform(int(rf_c.predict(inpredict)[0])).encode('ascii')

'Copenhagen'

Finally, Lets package our model and save it to object storage

In [31]:
filename = os.path.join(mkdtemp(), 'movie')
joblib.dump(rf_c, filename + '_c.gz', compress=3)
joblib.dump(d, filename + '_d.gz', compress=3)
conn = swiftclient.Connection(key= objs['password'],authurl='https://identity.open.softlayer.com/v3',auth_version='3',
os_options={"project_id": objs['project_id'],"user_id": objs['user_id'],"region_name": objs['region']})
conn.put_object("Analytics",'movie_c.gz',open(filename + '_c.gz').read(),content_type='application/python-pickle')
conn.put_object("Analytics",'movie_d.gz',open(filename + '_d.gz').read(),content_type='application/python-pickle')

'258164c87ac08142393ac5570c2d3649'